# Task1_CNN_LSTM

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-08-10_16-28-50/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-08-10_16-28-50

## Best metric (RMSE)

203.214

0.927

In [4]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.CNN_LSTM'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [5]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='metric',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='metric',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='metric',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-08-10 16:28:50,387] A new study created in memory with name: optuna


In [6]:
results = tuner.fit()

2023-08-10 16:28:50,426	INFO wandb.py:320 -- Already logged into W&B.


(_WandbLoggingActor pid=66571) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin


Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_072855d3,2023-08-10_17-05-07,True,DESKTOP-0P789CI,100,4.76296,727.379,1.31197,4.0234e+17,0.64827,172.26.215.93,85148,2.33887,593.069,311.39,3.64825,311.39,1691654707,0.948191,0.27898,1.88291e+14,1.86594e+14,100,072855d3,0.434802,0.213468
FSR_Trainable_07769fcd,2023-08-10_17-15-53,True,DESKTOP-0P789CI,64,4.90523,768.795,1.2966,6.07742e+17,0.641753,172.26.215.93,89395,2.38966,572.397,100.948,1.36824,100.948,1691655353,0.979046,0.299197,1.67269e+14,3.00202e+14,64,07769fcd,0.440279,0.201474
FSR_Trainable_0a4bcaa3,2023-08-10_16-45-12,True,DESKTOP-0P789CI,4,5.35259,1472.15,1.34902,1.7415e+18,0.859286,172.26.215.93,78107,2.63176,1133.91,29.2114,6.94429,29.2114,1691653512,1.07617,0.507983,2.01318e+14,5.71236e+14,4,0a4bcaa3,0.499358,0.359929
FSR_Trainable_0ca736a5,2023-08-10_16-33-38,True,DESKTOP-0P789CI,8,5.76648,1466.3,1.44054,1.72149e+18,0.860441,172.26.215.93,73070,2.68527,1124.72,28.9223,3.69158,28.9223,1691652818,1.13188,0.503006,1.86061e+14,5.24718e+14,8,0ca736a5,0.499558,0.360883
FSR_Trainable_0cc277c6,2023-08-10_16-49-31,True,DESKTOP-0P789CI,1,4.60991,1467.65,1.33387,6.97434e+16,21.1816,172.26.215.93,79985,2.44394,1182.68,3.77236,3.77236,3.77236,1691653771,12.6093,11.3433,3.93598e+15,2.48341e+15,1,0cc277c6,7.77869,13.403
FSR_Trainable_0d2c4fda,2023-08-10_16-43-00,True,DESKTOP-0P789CI,32,4.88406,1511.92,1.30092,2.12958e+18,0.820496,172.26.215.93,76867,2.48398,1112.59,85.4892,2.70097,85.4892,1691653380,0.987978,0.525306,2.06775e+14,7.22996e+14,32,0d2c4fda,0.467745,0.352751
FSR_Trainable_0f412368,2023-08-10_16-51-37,True,DESKTOP-0P789CI,16,5.11881,1491.87,1.30753,1.94562e+18,0.831473,172.26.215.93,81104,2.53279,1120.09,55.647,3.33329,55.647,1691653897,1.03144,0.519786,2.05764e+14,6.70248e+14,16,0f412368,0.475816,0.355657
FSR_Trainable_10524936,2023-08-10_16-43-45,True,DESKTOP-0P789CI,1,5.13763,1341.89,1.32825,3.84483e+17,20.9139,172.26.215.93,77623,2.5158,1086.76,3.24751,3.24751,3.24751,1691653425,15.6496,12.8625,1.24604e+16,1.61323e+16,1,10524936,8.42552,12.4884
FSR_Trainable_15f12775,2023-08-10_17-15-22,True,DESKTOP-0P789CI,1,4.44401,1400.2,1.13329,5.53242e+16,24.2141,172.26.215.93,90523,2.46332,1165.69,2.41903,2.41903,2.41903,1691655322,15.532,10.4397,5.42695e+15,2.51261e+15,1,15f12775,11.3314,12.8827
FSR_Trainable_19390f08,2023-08-10_17-11-05,True,DESKTOP-0P789CI,100,4.60737,663.756,1.17506,5.61214e+17,0.601072,172.26.215.93,87767,2.32711,498.493,146.652,1.58454,146.652,1691655065,0.921944,0.255959,1.64635e+14,2.66787e+14,100,19390f08,0.429665,0.171407


2023-08-10 16:29:01,150	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 1.774 s, which may be a performance bottleneck.
2023-08-10 16:29:01,156	WARNING util.py:315 -- The `process_trial_result` operation took 1.780 s, which may be a performance bottleneck.
2023-08-10 16:29:01,157	WARNING util.py:315 -- Processing trial results took 1.781 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-08-10 16:29:01,158	WARNING util.py:315 -- The `process_trial_result` operation took 1.783 s, which may be a performance bottleneck.
(_WandbLoggingActor pid=66571) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=66571) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=66571) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=66571) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-10_16-28-50/FSR_Trainable_26b73957_1_cr